In [22]:
import requests
from langchain.tools import tool

@tool
def get_ethusdt_avg_price() -> str:
    """Получить среднюю цену ETHUSDT (avgPrice за последние 5 минут) из Binance."""
    url = "https://api.binance.com/api/v3/avgPrice?symbol=ETHUSDT"
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    data = r.json()
    return data["price"]


In [42]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    api_key="",
    base_url="http://127.0.0.1:11434/v1",
    model="llama3-groq-tool-use:8b",
    temperature=0.0,
    max_tokens=1000
)

agent = create_agent(
    model=model,
    tools=[get_ethusdt_avg_price],
    system_prompt=(
        "Ты помощник. Если пользователь спрашивает цену ETHUSDT — "
        "используй инструмент get_ethusdt_avg_price. "
        "Ответь одной короткой фразой с ценой."
    ),
)


In [43]:
from langchain.messages import HumanMessage


result = agent.invoke({
    "messages": [
        HumanMessage(content="Сколько сейчас ETHUSDT?"),
    ]
})

print(result["messages"][-1].content)

The current average price of ETHUSDT is $2945.07.


In [44]:
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Сколько сейчас стоит ETHUSDT?"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Сколько сейчас стоит ETHUSDT?
================================== Ai Message ==================================
Tool Calls:
  get_ethusdt_avg_price (call_2sti3wid)
 Call ID: call_2sti3wid
  Args:
================================= Tool Message =================================
Name: get_ethusdt_avg_price

2945.07363554
================================== Ai Message ==================================

The current average price of ETHUSDT is $2945.07.


In [45]:
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Сколько Как у тебя дела?"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Сколько Как у тебя дела?
================================== Ai Message ==================================

I'm doing well, thank you! How can I assist you today?
